In [ ]:
import pandas as pd 
import os 
import pathlib 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

path_to_main_output = "/media/hieunguyen/HNHD01/outdir/ampliconSeq/new/data_analysis"

path_to_02_output = os.path.join(path_to_main_output, "02_output")
path_to_03_output = os.path.join(path_to_main_output, "03_output")
os.system(f"mkdir -p {path_to_03_output}")

umt_distance = 1
inputdir = f"/media/hieunguyen/HNHD01/outdir/ampliconSeq/from_VM/UMT_DISTANCE_{umt_distance}"

samplesheet = pd.read_csv(os.path.join("../experiments/SampleSheet_UMI_runs.csv"))


all_unmapped_bams = pd.DataFrame(
    data = [str(item) for item in pathlib.Path(inputdir).glob("BISMARK_ALIGNMENT_UNMAPPED_BAM/*.bam")],
    columns = ["path"]
)
all_unmapped_bams["SampleID"] = all_unmapped_bams["path"].apply(lambda x: x.split("/")[-1].split("_R1")[0])

##### connor fastqs
all_connor_outputs = pd.DataFrame(
    data = [str(item) for item in pathlib.Path(inputdir).glob("CONNOR_UMI_OUTPUT/*.connor_R1.fastq.gz")],
    columns = ["path"]
)
all_connor_outputs["SampleID"] = all_connor_outputs["path"].apply(lambda x: x.split("/")[-1].split(".connor")[0])
all_connor_outputs["countFASTQ1"] = all_connor_outputs["path"].apply(
    lambda x: int(os.popen(f"zcat '{x}' | wc -l").read().strip())
)
all_connor_outputs["countFASTQ2"] = all_connor_outputs["path"].apply(
    lambda x: int(os.popen(f"zcat '{x.replace("R1", "R2")}' | wc -l").read().strip())
)

##### connor fully annotated bams
all_annotated_bam = pd.DataFrame(
    data = [str(item) for item in pathlib.Path(inputdir).glob("CONNOR_UMI_OUTPUT/*fully_annotated.bam")],
    columns = ["annotated_bam_path"]
)
all_annotated_bam["SampleID"] = all_annotated_bam["annotated_bam_path"].apply(lambda x: x.split("/")[-1].split(".connor")[0])

##### merge connor fastq and annotated bam dataframe
all_connor_outputs = all_connor_outputs.merge(
    all_annotated_bam, on="SampleID", how="left"
)

all_connor_cov = pd.DataFrame(
    data = [str(item) for item in pathlib.Path(inputdir).glob("BISMARK_COV/*connor*.cov")],
    columns = ["path"]
)
all_connor_cov["SampleID"] = all_connor_cov["path"].apply(lambda x: x.split("/")[-1].split(".connor")[0])


samplesheet["1_unmapped_bam"] = samplesheet["SampleID"].apply(lambda x: "yes" if x in all_unmapped_bams.SampleID.unique() else "no")
samplesheet["2_connor_fastq"] = samplesheet["SampleID"].apply(lambda x: "yes" if x in all_connor_outputs.SampleID.unique() else "no")
samplesheet["3_connor_cov"] = samplesheet["SampleID"].apply(lambda x: "yes" if x in all_connor_cov.SampleID.unique() else "no")

# save missing samples
samplesheet[(samplesheet["1_unmapped_bam"] == "no")].to_csv(f"../experiments/missing_for_umt_distance_{umt_distance}.csv", index=False)